### [1970-你能穿过矩阵的最后一天](https://leetcode.cn/problems/last-day-where-you-can-still-cross/description/?envType=daily-question&envId=2025-12-29)

<font color=red>困难</font>  

给你一个下标从 1 开始的二进制矩阵，其中 0 表示陆地，1 表示水域。同时给你 row 和 col 分别表示矩阵中行和列的数目。
一开始在第 0 天，整个 矩阵都是 陆地 。但每一天都会有一块新陆地被 水 淹没变成水域。给你一个下标从 1 开始的二维数组 cells ，其中 cells[i] = [ri, ci] 表示在第 i 天，第 ri 行 ci 列（下标都是从 1 开始）的陆地会变成 水域 （也就是 0 变成 1 ）。
你想知道从矩阵最 上面 一行走到最 下面 一行，且只经过陆地格子的 最后一天 是哪一天。你可以从最上面一行的 任意 格子出发，到达最下面一行的 任意 格子。你只能沿着 四个 基本方向移动（也就是上下左右）。
请返回只经过陆地格子能从最 上面 一行走到最 下面 一行的 最后一天 。

<font color=#00FF00>类似题</font>


[Solution 803](Solution803.ipynb)

### 解法1 - 并查集

本质上是判断最上边和最下边是否连通，可以自然的想到使用 <font color=#00CCFF>并查集</font>算法 。 并查集的特点是容易加边，不容易删边。
考虑反过来看，删边就变成了加边(水变成陆地)


In [2]:
class UnionFind:
    def __init__(self, n: int):
        # 一开始有n个互不连接的节点 [0, 1, 2, ... n-1]，下标表示自己，元素内容表示自己的父节点，一开始互不相连，父节点是自己
        self._fa = list(range(n))

    # 查找x的父节点
    # 同时做路径压缩，把x所在集合的所有元素的父亲节点都改成一样的
    def find(self, x: int) -> int:
        fa = self._fa
        if fa[x] != x:
            fa[x] = self.find(fa[x])  # 路径压缩,递归的找父节点
        return fa[x]

    def is_union(self, x: int, y: int) -> bool:
        # 判断两个节点是否相交
        return self.find(x) == self.find(y)

    def merge(self, from_: int, to_: int) -> None:
        x, y = self.find(from_), self.find(to_)
        self._fa[x] = y  # 将两个节点合并

有了上面的并查集的定义，就可以开始利用并查集来解题了
注意这里使用了一种常见的二维数组转一维数组的方法: `cell[x][y]` -> `slot[x * col + y]`

In [12]:
from typing import List


class Solution:
    def latestDayToCross(self, row: int, col: int, cells: List[List[int]]) -> int:
        directions = (0, -1), (0, 1), (-1, 0), (1, 0)

        top = row * col
        bottom = row * col + 1
        ufs = UnionFind(row * col + 2)

        land = [[False] * col for _ in range(row)]

        # 逆序遍历cells
        for day in range(len(cells) - 1, -1, -1):
            x, y = [i - 1 for i in cells[day]]  # 减去1是为了还原为数组的下标
            pos = x * col + y
            land[x][y] = True

            if x == 0:
                ufs.merge(pos, top)

            if x == row - 1:
                ufs.merge(pos, bottom)

            for dx, dy in directions:
                xx, yy = x + dx, y + dy
                if 0 <= xx < row and 0 <= yy < col and land[xx][yy]:
                    ufs.merge(pos, xx * col + yy)

            if ufs.is_union(top, bottom):
                return day

In [13]:
if __name__ == '__main__':
    print(Solution().latestDayToCross(2, 2, [[1, 1], [2, 1], [1, 2], [2, 2]]))

1   1
0   1
2


### 解法2 - 深度优先遍历
<font color=#00CCFF>感染模型</font> 想象最上边有病毒，要感染到最下边。  
在倒着遍历`cells`，把水变成陆地的过程中：  
如果陆地在第一行，那么被感染。  
如果陆地上下左右是水，或者未被感染的陆地，那么这个陆地未被感染。  
如果陆地与被感染的陆地相邻，那么也被感染。⚠注意：如果 `(x,y)` 从水变成陆地后，把被感染的陆地与未被感染的陆地连起来，那么我们还需要从 `(x,y)` 出发，`DFS` 访问其余未被感染的陆地，把访问到的陆地也标记为被感染。  
作者：灵茶山艾府  
[链接](https://leetcode.cn/problems/last-day-where-you-can-still-cross/solutions/936629/dao-xu-bing-cha-ji-by-endlesscheng-canj/)



In [6]:
class Solution:
    def latestDayToCross(self, row: int, col: int, cells: List[List[int]]) -> int:
        directions = (1, 0), (-1, 0), (0, 1), (0, -1)

        # 陆地标识， 0 - 水，1 - 未感染陆地, 2 - 已感染陆地
        land = [[0] * col for _ in range(row)]

        # 从 x,y 出发，是否能到达顶部
        def can_reach_from_top(x: int, y: int) -> bool:
            if x == 0:
                return True
            for dx, dy in directions:
                # 如果从(x,y)出发能到达已感染陆地，则能到达顶部
                if 0 <= x + dx < row and 0 <= y + dy < col and land[x + dx][y + dy] == 2:
                    return True
            return False

        # 从(x,y)出发，是否能到达最后一行?
        # 这个方法用于：
        # 1. 当(x,y)可以连通顶部的时候(已感染的时候)，判断是否能到底部。
        # 2. DFS 遍历的过程中，将能感染的陆地标记为已感染
        def dfs(x: int, y: int) -> bool:
            if x == row - 1:
                return True
            land[x][y] = 2  # 感染当前陆地
            for dx, dy in directions:
                # 注意看这里的写法，dfs写在条件里了，这样写是不对的，注意区别
                #  if 0 <= x + dx < row and 0 <= y + dy < col and land[x + dx][y + dy] == 1 :
                #    return dfs(x + dx, y + dy)
                if 0 <= x + dx < row and 0 <= y + dy < col and land[x + dx][y + dy] == 1 and dfs(x + dx, y + dy):
                    return True
            return False

        for day in range(len(cells) - 1, -1, -1):
            x, y = [i - 1 for i in cells[day]]  # 下标转换
            land[x][y] = 1
            if can_reach_from_top(x, y) and dfs(x, y):
                return day


if __name__ == '__main__':
    print(Solution().latestDayToCross(row=3, col=3,
                                      cells=[[1, 2], [2, 1], [3, 3], [2, 2], [1, 1], [1, 3], [2, 3], [3, 2], [3, 1]]))

3
